# Stacked Denoising Autoencoder regression

Import libraries

In [29]:
import numpy as np
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense, Activation, GaussianNoise, LeakyReLU, Dropout
from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print(tensorflow.__version__)

2.1.0


In [2]:
#Checking whether your machine running only on CPU or with GPU

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1708641289864651689
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4930941747
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7906018761298007651
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


Read and load datasets

In [3]:
# load dataset
dataframe = pandas.read_csv("flesh_color_gene.txt", delim_whitespace=True, header=None,)
dataset = dataframe.values
print("data shape :", dataframe.shape)
data_len = dataset.shape[1]
print("no. of features : ", data_len )

data shape : (86, 15063)
no. of features :  15063


Splitting the dataset into taining and testing set

In [4]:
# split into input (X) and output (Y) variables
train, test = train_test_split(dataset, test_size=0.2)
trainX = train[:,0:data_len-1]
trainY = train[:,data_len-1]
testX = test[:,0:data_len-1]
testY = test[:,data_len-1]

print("trainX : ", trainX.shape)
print("trainY : ", trainY.shape)
print("testX : ", testX.shape)
print("testY : ", testY.shape)
#np.savetxt('Y.txt', trainY)
#np.savetxt('Y_test.txt', testY)

trainX :  (68, 15062)
trainY :  (68,)
testX :  (18, 15062)
testY :  (18,)


Normalizing the data

In [5]:
# Normalizing train data

max_abs_scaler = MaxAbsScaler()
normX = max_abs_scaler.fit_transform(trainX)
input_shape = normX.shape
#print("X norm : ", normX[:,15439])
print("normX shape : ", input_shape)
#np.savetxt('xnorm.txt', normX)
small_train = normX[:, 0:8000]

# Normalizing test data

max_abs_scaler = MaxAbsScaler()
normXt = max_abs_scaler.fit_transform(testX)
#np.savetxt('test_set.txt', normXt)
print("X norm shape :", normXt.shape)
small_test = normXt[:, 0:8000]

print("small train : ", small_train.shape)
print("smnall test : ", small_test.shape)

normX shape :  (68, 15062)
X norm shape : (18, 15062)
small train :  (68, 8000)
smnall test :  (18, 8000)


In [6]:
earlyStopping = EarlyStopping(monitor='loss', patience=50)
#K.set_learning_phase(1)
checkpointer = ModelCheckpoint(filepath='flesh_color.h5', verbose=1, save_best_only=True)
input_dims = data_len-1

# 1 - Simple Denoising Autoencoder

COMPULSORY!!! run the function below before go to the Stacked Denoising Autoencoder

Since my GPU can support only up to 6GB of VRAM (NVIDIA GTX 1060ti), I set only run on GPU if the features is less than 10,000. You can set it based on your machine capability

In [7]:
def dae (inputX, input_dims, output_dims, epoch, activation, loss, opti):
    
    #input_dims = inputX.shape
    #print("input dims : ", input_dims)
        
    model = Sequential()
    
    if input_dims > 9999:
        with tensorflow.device('/cpu:0'):
            print("Using CPU....")
            #model.add(Dense(input_dims, input_dim = input_dims))
            #model.add(GaussianNoise(0.5), input_shape=(input_dims, ))
            model.add(Dense(input_dims, input_dim = input_dims))
            model.add(GaussianNoise(0.5))
            model.add(Dense(output_dims, activation= activation, kernel_regularizer = regularizers.l1(0.01)))
            model.add(Dense(input_dims, activation= activation))
            model.compile(loss = loss, optimizer = opti)
            model.fit(inputX, inputX, epochs = epoch, batch_size = 16)
            model.summary()
    else:
        with tensorflow.device('/device:GPU:0'):
            print("Using GPU....")
            #model.add(GaussianNoise(0.5), input_shape=(input_dims, ))
            model.add(Dense(input_dims, input_dim = input_dims))
            model.add(GaussianNoise(0.5))
            model.add(Dense(output_dims, activation= activation, kernel_regularizer = regularizers.l1(0.01)))
            model.add(Dense(input_dims, activation= activation))
            model.compile(loss = loss, optimizer = opti)
            model.fit(inputX, inputX, epochs = epoch, batch_size = 16)
            model.summary()
    
    return model
    

Don't need to run the cells below if you want to go straight to the Stacked Denoising Autoencoder. This code is to show how the Denoising Autoencoder works.

In [8]:
autoencoder = dae(small_train, 
                  input_dims = 8000, 
                  output_dims = 500, 
                  epoch = 3, 
                  activation = 'relu', 
                  loss = 'mse', 
                  opti = 'adam',
                 )

Using GPU....
Train on 68 samples
Epoch 1/3
68/68 [==============================] - 1s 10ms/sample - loss: 525.1592
Epoch 2/3
68/68 [==============================] - 0s 2ms/sample - loss: 417.7400
Epoch 3/3
68/68 [==============================] - 0s 2ms/sample - loss: 352.9378
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8000)              64008000  
_________________________________________________________________
gaussian_noise (GaussianNois (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               4000500   
_________________________________________________________________
dense_2 (Dense)              (None, 8000)              4008000   
Total params: 72,016,500
Trainable params: 72,016,500
Non-trainable params: 0
___________________________

In [9]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8000)              64008000  
_________________________________________________________________
gaussian_noise (GaussianNois (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               4000500   
_________________________________________________________________
dense_2 (Dense)              (None, 8000)              4008000   
Total params: 72,016,500
Trainable params: 72,016,500
Non-trainable params: 0
_________________________________________________________________


# 2 - Stacked Denoising Autoencoder - pretrain

In [10]:
# Hyper-parameter

layers = [8000, 5000, 3000, 1000, 500, 300, 100] # Setting the size of your layer
epoch = 3
optimizer = 'adamax'
activation = 'relu'
loss = 'mse'

In [11]:
def sdae_pretrain (inputX, layers, activation, epoch, optimizer, loss):
    
    encoder = []
    decoder = []
    ae = []
    
    for i in range(len(layers)-1):
            # Greedily train each layer
            print("Now pretraining layer {} [{}-->{}]".format(i+1, layers[i], layers[i+1]))

            input_dims = layers[i]
            output_dims = layers[i+1]
            
            autoencoder = dae(inputX, input_dims, output_dims, epoch, activation, loss, optimizer)
            
            enc = Sequential()
            enc.add(Dense(output_dims, input_dim=input_dims))
            enc.compile(loss=loss, optimizer=optimizer)
            enc.layers[0].set_weights(autoencoder.layers[2].get_weights())
            inputX = enc.predict(inputX)
            print("check dimension : ", inputX.shape)
            enc.summary()
            #print(i)
            encoder.append(autoencoder.layers[2].get_weights())
            decoder.append(autoencoder.layers[3].get_weights())
            ae.append(autoencoder)

    
    return ae

In [12]:
test = sdae_pretrain(small_train, layers = layers, activation = activation, epoch = epoch, optimizer = optimizer, loss = loss)

Now pretraining layer 1 [8000-->5000]
Using GPU....
Train on 68 samples
Epoch 1/3
68/68 [==============================] - 1s 13ms/sample - loss: 3911.8391
Epoch 2/3
68/68 [==============================] - 0s 5ms/sample - loss: 2990.0804
Epoch 3/3
68/68 [==============================] - 0s 5ms/sample - loss: 2408.30780s - loss: 2516.59 - ETA: 0s - loss: 2467.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8000)              64008000  
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 8000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 5000)              40005000  
_________________________________________________________________
dense_5 (Dense)              (None, 8000)              40008000  
Total params: 144,02

68/68 [==============================] - 0s 4ms/sample - loss: 18.0575
Epoch 2/3
68/68 [==============================] - 0s 191us/sample - loss: 16.6350
Epoch 3/3
68/68 [==============================] - 0s 206us/sample - loss: 15.2901
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 300)               90300     
_________________________________________________________________
gaussian_noise_6 (GaussianNo (None, 300)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_25 (Dense)             (None, 300)               30300     
Total params: 150,700
Trainable params: 150,700
Non-trainable params: 0
_________________________________________________________________
check di

In [18]:
len(test)

6

OPTIONAL: Saving the models for each layer

In [21]:
for i, m in enumerate(test):
    filename="pre_train_l1" + str(i) + ".hd5"
    m.save(filename)

INFO:tensorflow:Assets written to: pre_train_l10.hd5\assets
INFO:tensorflow:Assets written to: pre_train_l11.hd5\assets
INFO:tensorflow:Assets written to: pre_train_l12.hd5\assets
INFO:tensorflow:Assets written to: pre_train_l13.hd5\assets
INFO:tensorflow:Assets written to: pre_train_l14.hd5\assets
INFO:tensorflow:Assets written to: pre_train_l15.hd5\assets


# 3 - Stacked Denoising Autoencoder - fine tuning

OPTIONAL: Load pre-trained model if you have one. Skip these 2 cells below if dont have one

In [23]:
test = []

with tensorflow.device('/cpu:0'):
    for i in range (len(layers)-1):
    
        test.append(load_model("pre_train_l1"+ str(i) + ".hd5"))

In [24]:
len(test)

6

Fine tuning the Autoencoders with the pre-trained model

In [37]:
def fine_tuning(weights, inputX, inputXt, inputY, inputYt, layers, epoch, activation, batch, optimizer, loss):

    encoder = []
    decoder = []

    for i in range(len(test)):
    
        encoder.append(test[i].layers[2].get_weights())
        decoder.append(test[i].layers[3].get_weights())
    
    with tensorflow.device('/device:gpu:0'): #I have to put this because the model size is to big for my GPU
        ft = Sequential()
        ft.add(Dense(layers[0], input_dim=layers[0]))
        ft.add(GaussianNoise(0.5))

        for i in range(len(layers)-1):
        #print(i)
            ft.add(Dense(layers[i+1], activation = activation, weights = encoder[i], kernel_regularizer = regularizers.l1_l2(0.01)))
        
        for i in reversed(range(len(layers)-1)):
        #print(i)
            ft.add(Dense(layers[i], activation = activation, weights = decoder[i]))
    ft.add(Dropout(0.2))
    ft.add(Dense(1000, activation = activation))
    ft.add(Dense(1, activation = 'sigmoid'))
    
    #ft.add(Dense(1, kernel_initializer = 'normal'))   
    ft.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    ft.fit(small_train, trainY, epochs = epoch, batch_size = batch, validation_data=(small_test, testY))
    ft.summary()
    
    return ft

In [38]:
tryla = fine_tuning(test, small_train, small_test, trainY, testY, layers = layers, epoch = 10, activation = 'relu', batch = 16, optimizer = 'adam', loss = 'binary_crossentropy')

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [ ]:
tryla.predict(x=normXt)

In [ ]:
tryla.save("fine_tune.hdf5")